In [1]:
!python -m pip install -q git+https://github.com/fbartolic/extending-jax.git

In [1]:
from jax.config import config

config.update("jax_enable_x64", True)

from jax import random
import jax.numpy as jnp

from ehrlich_aberth_jax import ehrlich_aberth_jax

/Users/fb90/miniforge3/envs/caustic/lib/python3.9/site-packages/jax/_src/lib/__init__.py:32: UserWarning: JAX on Mac ARM machines is experimental and minimally tested. Please see https://github.com/google/jax/issues/5501 in the event of problems.
  warnings.warn("JAX on Mac ARM machines is experimental and minimally tested. "


ImportError: dlopen(/Users/fb90/miniforge3/envs/caustic/lib/python3.9/site-packages/ehrlich_aberth_jax/cpu_ops.cpython-39-darwin.so, 0x0002): tried: '/Users/fb90/miniforge3/envs/caustic/lib/python3.9/site-packages/ehrlich_aberth_jax/cpu_ops.cpython-39-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e')), '/usr/local/lib/cpu_ops.cpython-39-darwin.so' (no such file), '/usr/lib/cpu_ops.cpython-39-darwin.so' (no such file)

In [2]:
p = jnp.array([
    -0.09681953 -0.00090856j,
    2.01207803 -0.0004709j,
    -1.12402336 + 0.00979704j,
    -1.54998502 -0.01104674j,
    1.1198831 + 0.00012531j,
    -0.00506219 + 0.00697788j,
])

coeffs = jnp.stack([p, p]).T
coeffs.shape

(6, 2)

In [3]:
ehrlich_aberth(coeffs)

NameError: name 'ehrlich_aberth' is not defined